##### Se requiere obtener informacion de las "peliculas" del año 2000 en adelante ordenado de manera descendete por la fecha de lanzamiento, se debe considerar su respectivo "genero" y su "idioma" que se encuentra en dicha pelicula.

1. De la pelicula se requiere mostrar el "titulo, el tiempo de duracion, fecha lanzamiento y el voto promedio"
2. Del idioma se requiere mostrar el  nombre del idioma
3. Del genero mostrar el nombre del genero

#### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
# Por ser carga incremental  debemos fitrar por file_date.
# Ya que esta particiondo por file_date
#movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake

movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
            .filter(f"file_date = '{v_file_date}'")

In [0]:
#language_df = spark.read.parquet(f"{silver_folder_path}/languages")

# Modulo: Delta Lake

language_df = spark.read.format("delta").load(f"{silver_folder_path}/languages")

In [0]:
# Por ser carga incremental  debemos fitrar por file_date.
# Ya que esta particiondo por file_date
#movies_languages_df = spark.read.parquet(f"{silver_folder_path}/movies_languages") \
#                     .filter(f"file_date = '{v_file_date}'")

# Modulo : Delta Lake

movies_languages_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_languages") \
                     .filter(f"file_date = '{v_file_date}'")

In [0]:
#genres_df = spark.read.parquet(f"{silver_folder_path}/genres")

# Modulo: Delta Lake
genres_df = spark.read.format("delta").load(f"{silver_folder_path}/genres")

In [0]:
# Por ser carga incremental  debemos fitrar por file_date.
# Ya que esta particiondo por file_date

# movies_genres_df = spark.read.parquet(f"{silver_folder_path}/movies_genres") \
#            .filter(f"file_date = '{v_file_date}'")

#Modulo: Delta Lake
movies_genres_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_genres") \
            .filter(f"file_date = '{v_file_date}'")

### Join "languages" y "movies_languages"

In [0]:
languages_mov_lan_df = language_df.join(movies_languages_df, 
                                        language_df.language_id == movies_languages_df.language_id,
                                        'inner') \
                      .select(language_df.language_name, language_df.language_id ,movies_languages_df.movie_id)

#### Join "genres" y "movies_genres"

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df,
                                    genres_df.genre_id == movies_genres_df.genre_id,
                                    "inner") \
                    .select(genres_df.genre_name, genres_df.genre_id ,movies_genres_df.movie_id)

#### Join "movies_df", "languages_mov_lan_df" y "genres_mov_lan_df"

- Filtrar las peliculas donde su fecha de lanzamiento sea mahor o igual a 2000

In [0]:
#Usamos la expresion SQL 
movie_filter_df = movies_df.filter("year_release_date >= 2000")

In [0]:
results_movie_genres_languages_df = movie_filter_df.join(languages_mov_lan_df,
                                    movie_filter_df.movie_id == languages_mov_lan_df.movie_id,
                                    "inner") \
                                                .join(genres_mov_gen_df,
                                        movie_filter_df.movie_id == genres_mov_gen_df.movie_id,
                                    "inner")

- Agregar la columna "created_date"

In [0]:
from pyspark.sql.functions import lit

In [0]:
results_df = results_movie_genres_languages_df \
             .select(movie_filter_df.movie_id,"language_id","genre_id","title","duration_time","release_date","vote_average","language_name","genre_name") \
             .withColumn("created_date", lit(v_file_date))

- Ordenar por la columna "release_date" de manera descendente

In [0]:
results_order_by_df = results_df.orderBy(results_df.release_date.desc())

In [0]:
display(results_order_by_df)

#### Escribir los datos en el DataLake en formato "Delta"

In [0]:
#overwrite_partition(results_order_by_df,"movie_gold","results_movie_genre_language","created_date")

In [0]:
# results_order_by_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_movie_genre_language")

# Crear una tabla en base al data frame results_order_by_df
# Creara la tabla results_movie_genre_language en a DB que se llame movie_gold
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Gold
#results_order_by_df.write.mode("overwrite").format("parquet").saveAsTable("movie_gold.results_movie_genre_language")

#Nota: Esto es una tabla administrada por Databricks, esto quiere decir que si la tabla se elimina, los registros tambien se van a eliminar
# Append, agrega los registros a la tabla existente, carga incremental de datos

#results_order_by_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_movie_genre_language")


#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.genre_id = src.genre_id AND tgt.created_date = src.created_date" # Buscar por particion
merge_delta_lake(results_order_by_df,"movie_gold","results_movie_genre_language",gold_folder_path,merge_condition,"created_date")




In [0]:
%sql
-- Consultar la DB movie_gold, la tabla results_movie_genre_language
SELECT created_date, COUNT(1) 
FROM movie_gold.results_movie_genre_language
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_movie_genre_language"))
display(spark.read.format("delta").load(f"{gold_folder_path}/results_movie_genre_language"))